In [70]:
import pandas as pd
import numpy as np

In [159]:
data_dict={'Users':['1','2','3','4','5','6'],
           'action1':[4.0,5.0,1.0,0.0,1.0,5.0],
           'action2':[5.0,3.0,0.0,2.0,0.0,1.0],
           'action3':[3.0,3.0,0.0,1.0,2.0,1.0],
           'romantic1':[0.0,2.0,4.0,4.0,3.0,2.0],
           'romantic2':[2.0,2.0,5.0,0.0,3.0,5.0],
           'romantic3':[1.0,0.0,4.0,3.0,4.0,0.0]
          }

In [160]:
data=pd.DataFrame(data_dict)

In [161]:
data=data.set_index('Users')

In [162]:
data=data

In [163]:
data

,action1,action2,action3,romantic1,romantic2,romantic3
Users,,,,,,
1,4.0,5.0,3.0,0.0,2.0,1.0
2,5.0,3.0,3.0,2.0,2.0,0.0
3,1.0,0.0,0.0,4.0,5.0,4.0
4,0.0,2.0,1.0,4.0,0.0,3.0
5,1.0,0.0,2.0,3.0,3.0,4.0
6,5.0,1.0,1.0,2.0,5.0,0.0


In [164]:
def standardize(row):
    return (row-row.mean()/row.max()-row.min())
    
data=data.apply(standardize,axis=1).T

In [165]:
data

Users,1,2,3,4,5,6
action1,3.5,4.5,0.533333,-0.416667,0.458333,4.533333
action2,4.5,2.5,-0.466667,1.583333,-0.541667,0.533333
action3,2.5,2.5,-0.466667,0.583333,1.458333,0.533333
romantic1,-0.5,1.5,3.533333,3.583333,2.458333,1.533333
romantic2,1.5,1.5,4.533333,-0.416667,2.458333,4.533333
romantic3,0.5,-0.5,3.533333,2.583333,3.458333,-0.466667


In [166]:
data.index

Index(['action1', 'action2', 'action3', 'romantic1', 'romantic2', 'romantic3'], dtype='object')

In [167]:
from scipy import sparse
sparse_df = sparse.csr_matrix(data)

In [168]:
sparse_df.toarray()

array([[ 3.5       ,  4.5       ,  0.53333333, -0.41666667,  0.45833333,
         4.53333333],
       [ 4.5       ,  2.5       , -0.46666667,  1.58333333, -0.54166667,
         0.53333333],
       [ 2.5       ,  2.5       , -0.46666667,  0.58333333,  1.45833333,
         0.53333333],
       [-0.5       ,  1.5       ,  3.53333333,  3.58333333,  2.45833333,
         1.53333333],
       [ 1.5       ,  1.5       ,  4.53333333, -0.41666667,  2.45833333,
         4.53333333],
       [ 0.5       , -0.5       ,  3.53333333,  2.58333333,  3.45833333,
        -0.46666667]])

In [169]:
from sklearn.metrics.pairwise import cosine_similarity
corrMatrix=pd.DataFrame(cosine_similarity(sparse_df))

In [170]:
corrMatrix

,0,1,2,3,4,5
0,1.000000,0.706308,0.783787,0.305357,0.687429,-0.005379
1,0.706308,1.000000,0.844652,0.152517,0.224206,0.042829
2,0.783787,0.844652,1.000000,0.310272,0.393614,0.209652
3,0.305357,0.152517,0.310272,1.000000,0.669840,0.840111
4,0.687429,0.224206,0.393614,0.669840,1.000000,0.525094
5,-0.005379,0.042829,0.209652,0.840111,0.525094,1.000000


In [171]:
corrMatrix=corrMatrix.set_index(data.index)

In [172]:
corrMatrix.columns=data.index

In [173]:
corrMatrix

,action1,action2,action3,romantic1,romantic2,romantic3
action1,1.000000,0.706308,0.783787,0.305357,0.687429,-0.005379
action2,0.706308,1.000000,0.844652,0.152517,0.224206,0.042829
action3,0.783787,0.844652,1.000000,0.310272,0.393614,0.209652
romantic1,0.305357,0.152517,0.310272,1.000000,0.669840,0.840111
romantic2,0.687429,0.224206,0.393614,0.669840,1.000000,0.525094
romantic3,-0.005379,0.042829,0.209652,0.840111,0.525094,1.000000


In [174]:
def get_similar_movies(movie_name,rating):
    relevant_movies=corrMatrix[movie_name]
    if rating>2.5:
        relevant_movies=corrMatrix[movie_name]
    else:
        relevant_movies=corrMatrix[movie_name]*(-1)
        
    return relevant_movies.sort_values(ascending=False)

In [175]:
get_similar_movies('action1',5)

action1      1.000000
action3      0.783787
action2      0.706308
romantic2    0.687429
romantic1    0.305357
romantic3   -0.005379
Name: action1, dtype: float64

In [178]:
action_lover = [("action1",5),("romantic2",1),("romantic3",1)]
similar_scores = pd.DataFrame()
for movie,rating in action_lover:
    similar_scores = similar_scores.append(get_similar_movies(movie,rating),ignore_index = True)
    print(similar_scores)

   action1   action2   action3  romantic1  romantic2  romantic3
0      1.0  0.706308  0.783787   0.305357   0.687429  -0.005379
    action1   action2   action3  romantic1  romantic2  romantic3
0  1.000000  0.706308  0.783787   0.305357   0.687429  -0.005379
1 -0.687429 -0.224206 -0.393614  -0.669840  -1.000000  -0.525094
    action1   action2   action3  romantic1  romantic2  romantic3
0  1.000000  0.706308  0.783787   0.305357   0.687429  -0.005379
1 -0.687429 -0.224206 -0.393614  -0.669840  -1.000000  -0.525094
2  0.005379 -0.042829 -0.209652  -0.840111  -0.525094  -1.000000
